In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('train.csv')
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [3]:
df.shape

(404290, 6)

In [4]:
new_df = df.sample(30000)

In [5]:
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [7]:
new_df = new_df.dropna(subset=['question1', 'question2']).reset_index(drop=True)

In [8]:
ques_df = new_df[['question1','question2']]
ques_df.head()

,question1,question2
0,How was the KVPY 2016 SA paper? What is the ex...,What do you think the cutoff of KVPY 2016 SA w...
1,Culturally how do we view failures?,How do we view failures culturally?
2,How do I start final year project?,How do I apply for doing 8th sem final year pr...
3,"How can I create a physical push button, that ...",How will I create a physical button that will ...
4,How much Benadryl can I give my dog for allerg...,Can I give my dog Benadryl for his allergies?


In [9]:
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
questions = list(ques_df['question1']) + list(ques_df['question2'])

In [11]:
cv = CountVectorizer(max_features=3000)

In [12]:
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [13]:
temp_df1 = pd.DataFrame(q1_arr, index = ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index = ques_df.index)

In [14]:
temp_df = pd.concat([temp_df1, temp_df2], axis = 1)

In [15]:
temp_df.shape

(30000, 6000)

In [16]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
temp_df['is_duplicate'] = new_df['is_duplicate']

temp_df.head()

# Random Forest

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
X_train,X_test,y_train,y_test = train_test_split(temp_df.iloc[:,0:-1].values, temp_df.iloc[:,-1].values, random_state=40)

In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [23]:
rf = RandomForestClassifier()

In [24]:
rf.fit(X_train, y_train)

RandomForestClassifier()

In [25]:
y_pred = rf.predict(X_test)

In [26]:
accuracy_score(y_test,y_pred)

0.7454666666666667

# XGBoost

In [30]:
from xgboost import XGBClassifier

In [31]:
xgb = XGBClassifier()

In [32]:
xgb.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [33]:
y_pred = xgb.predict(X_test)

In [35]:
accuracy_score(y_test, y_pred)

0.7389333333333333